# Sparse network asymptotics for logistic regression under possible misspecification: Monte Carlo experiments
_September 2022 (Revised July 2024)_   
Bryan S. Graham, UC - Berkeley, bgraham@econ.berkeley.edu

This iPython Jupyter notebook reproduces the Monte Carlo results reported in the Appendix C of the published version of my paper "Sparse network asymptotics for logistic regression under possible misspecification.” 

The scripts below were written for Python 3.6 and revised to execute properly in Python 3.9. The Anaconda distribution of Python, available at https://www.continuum.io/downloads, comes bundled with all the scientific computing packages used in this notebook. The notebook additionally uses the _ipt_ and _netrics_ modules that I have created. They are available on my GitHub page (https://github.com/bryangraham); users will need to download these packages to execute the code below. Details on how to do this are provided in the markdown boxes below.


Please feel free to use and modify the material in this notebook for your own research purposes. All I ask is that you cite both the underlying research as well as this codebase (see below for a suggested citation). If you find any errors in what follows I would be happy to hear about them. While I am not able to provide meaningful support for potential users of this code, I am willing to answer questions when/if I have the bandwidth to so.

**References:**

Graham, Bryan S. (2020). "Sparse network asymptotics for logistic regression under possible misspecification,” _CEMMAP Working Paper CWP51/20_ (Revised 2024).   

**Suggested code citation:**  

Graham, Bryan S. (2022). "Sparse network asymptotics for logistic regression under possible misspecification: Monte Carlo experiments Python Jupyter notebook," (Version 1.0) [Computer program]. Available at http://bryangraham.github.io/econometrics/ (Accessed 20 September 2022).

In [12]:
# Direct Python to plot all figures inline (i.e., not in a separate window)
%matplotlib inline

# Main scientific computing modules
# Load library dependencies
import time
import numpy as np
import scipy as sp
import pandas as pd
import itertools as it

# Import matplotlib 
import matplotlib.pyplot as plt

# networkx module for the analysis of network data
import networkx as nx

Print out the version of Python used as well as the version numbers of key scientific computing modules.

In [13]:
import sys

print("Version of Python used:")
print(sys.version)
print("")
print("numpy version:     " + np.__version__)
print("scipy version:     " + sp.__version__)
print("pandas version:    " + pd.__version__)

Version of Python used:
3.9.13 (main, Aug 25 2022, 18:29:29) 
[Clang 12.0.0 ]

numpy version:     1.21.5
scipy version:     1.9.1
pandas version:    1.4.4


Load the **netrics** package. The Python 2.7 version of this package is registered on [PyPi](https://pypi.python.org/pypi/netrics/), with a GitHub repository at https://github.com/bryangraham/netrics_py27. For an informal introduction to the package see this [blog post](http://bryangraham.github.io/econometrics/networks/2016/09/15/netrics-module.html). The blog post also includes links to additional resources. The Python 3.6 version of the package is still under development, but currently available code can be found on GitHub at https://github.com/bryangraham/netrics. Once a basic level of functionality and reliability is in place I will register it on PyPi. The **netrics** package uses some functionality from the **ipt** package; so this latter package is loaded as well. You can read more about the **ipt** package at this [blog post](http://bryangraham.github.io/econometrics/causal/inference/2016/05/15/IPT-module.html).
<br>
<br>
To execute the code in this notebook (i) download the _ipt_ and _netrics_ packages from Github onto your local computer and (ii) adjust the directory locations in the two "sys.path.append" commands in the code block below to point to the location of the two downloaded packages.

In [14]:
# Append location of netrics and ipt modules base directory to system path
# NOTE: only required if permanent install not made (see comments above)
import sys
sys.path.append('/Users/bgraham/Dropbox/Sites/software/ipt/')
sys.path.append('/Users/bgraham/Dropbox/Sites/software/netrics/')

# Load ipt and netrics modules
import ipt as ipt
import netrics as netrics

### Monte Carlo Design

(NOTE: Some of the text in this markdown box is borrowed from the paper.)   

For the Monte Carlo experiments I set the graphon, $h_{n}\left(\mu,W_{i},X_{j},A_{i},B_{j},V_{ij}\right)$, equal to 

$$ Y_{ij}	=1\left(\alpha+z\left(W_{i},X_{j}\right)'\beta+\ln\left(A_{i}\right)+\ln\left(B_{j}\right)-\ln\left(n\right)\geq V_{ij}\right) $$ 

with $V_{ij}$ a standard exponential random variable.   

Averaging over $V_{ij}$ yields

$$ \mathbb{E}_{n}\left[\left.Y_{ij}\right|W_{i},X_{j},A_{i},B_{j}\right]=\frac{1}{n}\exp\left(\alpha+z\left(W_{i},X_{j}\right)'\beta\right)A_{i}B_{j}. $$ 

I set $\left\{ A_{i}\right\} _{i=1}^{N}$ and $\left\{ B_{j}\right\} _{j=1}^{M}$ to be iid log-normal sequences of random variables with $\mu=-1/12$ and $\sigma=1/\sqrt{6}$. This implies that both $A_{i}$ and $B_{j}$ are mean one and, furthermore, that the variance of the sum $\ln\left(A_{i}\right)+\ln\left(B_{j}\right)$ is one third that of $V_{ij}$. This generates meaningful, but not overpowering, cross dyad dependence. Under these assumptions the regression function equals

$$g_{n}\left(w,x\right)=\frac{1}{n}\exp\left(\alpha+z\left(W_{i},X_{j}\right)'\beta\right).$$

Finally I set $z\left(W_{i},X_{j}\right)=\left(\begin{array}{ccc}
W_{i} & X_{j} & W_{i}X_{j}\end{array}\right)'$ with $\left\{ W_{i}\right\} _{i=1}^{N}$ iid Bernouli with a success probability $\pi_{w}=1/\sqrt{3}$ and $\left\{ X_{j}\right\} _{j=1}^{M}$ iid Bernouli with a success probability $\pi_{x}=1/\sqrt{3}$. This implies that one third of dyads are of the $W_{i}=X_{j}=1$ type.

I simulate data for five sampe sizes: $n=64, 144, 256, 576$ and $1024$ with $N=M$ in all cases. I set $\alpha=\ln\left(64\times0.04\right)$, $\beta_{w}=\beta_{x}=0$ and $\beta_{wx}=\ln4\approx1.3863$. This implies that $\rho_{n}=0.08, 0.036, 0.020, 0.009$ and $0.005$ across the five designs. For each design I perform $5,000$ Monte Carlo replications.

The design is stylized enough that the various score projections appearing in the main paper (see Equations (21) to (27) in the paper) can be computed analytically.

Specifically we have a "score" equation of, maintaining the notation defined in the paper,

$$ s_{ij,n}=\left(Y_{ij}-e_{ij,n}\right)R_{ij}. $$

Averaging over $V_{ij}$ gives a first projection of   

$$ \bar{s}_{ij,n}=\left(\frac{1}{n}\exp\left(\alpha+z\left(W_{i},X_{j}\right)'\beta\right)A_{i}B_{j}-e_{ij,n}\right)R_{ij}. $$

This term is the kernel of the U-Statistic discussed in the paper. The two terms which together define the Hajek projection of this U-Statistic are


$$\bar{s}_{1i,n}^{c}=\left(1-\pi_{x}\right)\left(\frac{1}{n}\exp\left(\alpha+W_{i}\beta_{w}\right)\left(A_{i}-\frac{1}{1+\frac{1}{n}\exp\left(\alpha+W_{i}\beta_{w}\right)}\right)\right)\left(\begin{array}{c}
1\\
W_{i}\\
0\\
0
\end{array}\right)
	+\pi_{x}\left(\frac{1}{n}\exp\left(\alpha+W_{i}\beta_{w}+\beta_{x}+W_{i}\beta_{wx}\right)\left(A_{i}-\frac{1}{1+\frac{1}{n}\exp\left(\alpha+W_{i}\beta_{w}+\beta_{x}+W_{i}\beta_{wx}\right)}\right)\right)\left(\begin{array}{c}
1\\
W_{i}\\
1\\
W_{i}
\end{array}\right) $$

and

$$ \bar{s}_{1j,n}^{p}=	\left(1-\pi_{w}\right)\left(\frac{1}{n}\exp\left(\alpha+X_{j}\beta_{x}\right)\left(B_{j}-\frac{1}{1+\frac{1}{n}\exp\left(\alpha+X_{j}\beta_{x}\right)}\right)\right)\left(\begin{array}{c}
1\\
0\\
X_{j}\\
0
\end{array}\right)
	+\pi_{w}\left(\frac{1}{n}\exp\left(\alpha+\beta_{w}+X_{j}\beta_{x}+X_{j}\beta_{wx}\right)\left(B_{j}-\frac{1}{1+\frac{1}{n}\exp\left(\frac{1}{n}\exp\left(\alpha+\beta_{w}+X_{j}\beta_{x}+X_{j}\beta_{wx}\right)\right)}\right)\right)\left(\begin{array}{c}
1\\
1\\
X_{j}\\
X_{j}
\end{array}\right).
$$

These terms can be used to compute $S_n$, $U_{1n}$, $U_{2n}$ and $V_{n}$ in "closed form". The variances of these terms can be computed using the sample variance across the $5,000$ Monte Carlo replications. **Table 4** in the paper (in Appendix C) examiness the properties of these "score" components in detail; verifying the rate-of-convergence calculations in the paper.

The bias term of $S_{n}$ is as given in the paper in equation (27). However the first term in this equation is zero for the design considered here because $\lambda \left(w,x\right)$ takes an exponential form (which is consistent with the limiting form of the logit function).

The next block of code executes the Monte Carlo simulations.

The output from this code block reproduces the analytic bias numbers in **Column 6 of Table 4** in the paper (respectively 2.101 and 1.081). Only the results from Designs 3 ($N=256$) and 5 ($N=1024$) below are report in the paper.

In [15]:
# N,M = 32, 72, 128, 288, 512, 1152, 2048 sqrt(n) = 8, 12, 16, 24, 32, 48, 64
# (simulated sample sizes and relative magnitudes in "rate-of-convergence" terms")

designs = [32, 72, 128, 288, 512]
NumDesigns = len(designs)

 # Number of Monte Carlo simulations
S = 5000               

#----------------------------------------------------#
#- CORE FEATURES OF MONTE CARLO DESIGNS             -#
#----------------------------------------------------#
    
n0 = 64                 # Initial sample size used to calibrate intercept, alpha

mu    = -1/12           # Mean and variance of consumer and product effects
sigma =  1/np.sqrt(6)   # Variance of ln(A_i) + ln(B_j) = 2sigma^2

pi_w  = (1/3)**(1/2)    # Covariate distribution parameters
pi_x  = (1/3)**(1/2)    # One third transaction opportunities are W_i=X_j=1 cases.

# Initialize matrices for storage of Monte Carlo results
theta_hat = np.zeros((S,NumDesigns))
coverage  = np.zeros((S,NumDesigns*4))
se_hat    = np.zeros((S,NumDesigns*4))

# Matrices to store the components of "score" vector
S_n       = np.zeros((S,NumDesigns*4))
U1_n      = np.zeros((S,NumDesigns*4))
U2_n      = np.zeros((S,NumDesigns*4))
V_n       = np.zeros((S,NumDesigns*4))


#----------------------------------------------------#
#- BEGIN MONTE CARLO SIMULATIONS.                   -#
#----------------------------------------------------#

for b in range(0,NumDesigns ):
    
    # Set random seed and start bth Monte Carlo experiments
    # (same seed is used for each design)
    np.random.seed(seed=361)
    
    print("Monte Carlo design " + '%.0f' % (b+1) + " of " + '%.0f' % NumDesigns )

    #----------------------------------------------------#
    #- DEFINE DATA GENERATING PROCESS FOR MONTE CARLO   -#
    #- MISPECIFIED CASE: "EXOBIT"                       -#
    #----------------------------------------------------#

    N = designs[b]                 # Number of consumers
    M = designs[b]                 # Number of products
    n = N + M                      # Standard error should decrease at rate 1/sqrt(n)
    
    # Parameters indexing regression function
    theta = np.array([0, 0, np.log(4), np.log(n0*0.04)])

    # Marginal purchase probability for current design, rho_n
    rho0_n   = (1 - pi_w) * (1 - pi_x) * (1/n)*np.exp(theta[3])+ \
               (1 - pi_w) * pi_x       * (1/n)*np.exp(theta[1] + theta[3]) + \
                     pi_w * (1 - pi_x) * (1/n)*np.exp(theta[0] + theta[3]) + \
                     pi_w * pi_x       * (1/n)*np.exp(theta[0] + theta[1] + theta[2] + theta[3]) 

    print('Number of consumers and products, n = N + M : ' + '%.0f' % n)
    print('Marginal purchase probability               : ' + '%.3f' % rho0_n)

    # Computed fixed n bias of the score vector, b_n(theta)
    b_n   = (1 - pi_w) * (1 - pi_x) * (1/n)*np.exp(theta[3]) * \
                                      (1 - 1/(1 +(1/n)*np.exp(theta[3]))) * np.array([0, 0, 0, 1]) + \
            (1 - pi_w) * pi_x       * (1/n)*np.exp(theta[1] + theta[3]) * \
                                      (1 - 1/(1 +(1/n)*np.exp(theta[1] + theta[3]))) * np.array([0, 1, 0, 1]) + \
                  pi_w * (1 - pi_x) * (1/n)*np.exp(theta[0] + theta[3]) * \
                                      (1 - 1/(1 +(1/n)*np.exp(theta[0] + theta[3]))) * np.array([1, 0, 0, 1]) + \
                  pi_w * pi_x       * (1/n)*np.exp(theta[0] + theta[1] + theta[2] + theta[3]) * \
                                      (1 - 1/(1 +(1/n)*np.exp(theta[0] + theta[1] + theta[2] + theta[3]))) \
                                    * np.array([1, 1, 1, 1])
    print('Bias in pseudo-score vector n^(3/2)*S_n     : ' + '%.3f' %  (n**(3/2)*b_n[2]))
    print('(Above bias is for the component of S_n corresponding to the interaction coefficient)')
    
    # Compute the limiting value of normalized/rescaled Hessian matrix (nH)
    GAMMA = (1 - pi_w) * (1 - pi_x) * np.exp(theta[3]) * \
                                      np.array([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 1]]) +\
                  pi_w * (1 - pi_x) * np.exp(theta[0] + theta[3]) * \
                                      np.array([[1, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 1]]) +\
            (1 - pi_w) *       pi_x * np.exp(theta[1] + theta[3]) * \
                                      np.array([[0, 0, 0, 0], [0, 1, 0, 1], [0, 0, 0, 0], [0, 1, 0, 1]]) +\
                  pi_w *       pi_x * np.exp(theta[0] + theta[1] + theta[1] + theta[2]) * \
                                      np.array([[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]])

    # Compute the inverse
    iGAMMA = np.linalg.inv(GAMMA)

    #----------------------------------------------#
    #- MONTE CARLO SIMULATIONS FOR CURRENT DESIGN -#
    #----------------------------------------------#

    for s in range(0,S):
        start = time.time()
    
        #-------------------------------------#
        #- STEP 1 : SIMULATE BIPARTITE GRAPH -#
        #-------------------------------------#
    
        # Simulate consumer and product heterogeneity/effects
        A = np.random.lognormal(mu, sigma, N)
        B = np.random.lognormal(mu, sigma, M)
    
        # Simulate consumer and product covariates
        W = np.random.binomial(1, pi_w, N)
        X = np.random.binomial(1, pi_x, M)
    
        # Generate s-th Monte Carlo draw of Y
        WX_n       = np.kron(W,X) 
    
        iota_M   = np.ones((M,))
        i        = np.kron(np.arange(0,N), iota_M)
        W_n      = np.kron(W, iota_M) 
        A_n      = np.kron(A, iota_M) 
    
        iota_N   = np.ones((N,))
        j        = np.kron(iota_N, np.arange(0,M))
        X_n      = np.kron(iota_N, X) 
        B_n      = np.kron(iota_N, B) 
    
        Y_n      = 1*(-theta[0]*W_n - theta[1]*X_n - theta[2]*WX_n - theta[3] \
                      - np.log(A_n) - np.log(B_n) + np.log(n) <= np.random.exponential(1, (N*M,)))
    
        # Mean of Y_ij given W_i, X_j, A_i and B_j
        hbar_n   = (1/n)*(np.exp(theta[0]*W_n + theta[1]*X_n + theta[2]*WX_n + theta[3])*A_n*B_n)
    
        # Logit function at "true" theta
        e_n      =  ((1/n)*np.exp(theta[0]*W_n + theta[1]*X_n + theta[2]*WX_n + theta[3]) / \
                 (1 + (1/n)*np.exp(theta[0]*W_n + theta[1]*X_n + theta[2]*WX_n + theta[3])))
    
        # Form regressor matrix as pandas dataframe for using in bilogit() command
        R = pd.DataFrame(np.vstack([Y_n, hbar_n, e_n, W_n, X_n, WX_n, i, j]).T, \
                         columns=['Y_ij', 'hbar_ij', 'e_ij', 'W_i', 'X_j', 'W_i x X_j', 'i', 'j'])    
        R = R.set_index(['i', 'j'], drop = True)  # Set dataframe multi-indexr
        R['constant'] = 1

        # Get outcome variable and drop from regressor matrix
        Y = R['Y_ij'].copy(deep=True)
        R.drop('Y_ij', axis=1, inplace=True)
    
        # Get expected outcome variable given W_i, X_j, A_i and B_j and drop from regressor matrix
        hbar_n = R['hbar_ij'].copy(deep=True)
        R.drop('hbar_ij', axis=1, inplace=True)
    
        # Get modeled expected outcome variable given W_i and X_j, anddrop from regressor matrix
        e_n = R['e_ij'].copy(deep=True)
        R.drop('e_ij', axis=1, inplace=True)
    
        #------------------------------------------#
        #- STEP 2 : CALCULATE SCORE DECOMPOSITION -#
        #------------------------------------------#
    
        # Score vector
        s_n =    (1/(N*M))*(R.to_numpy().T @ (Y - e_n).to_numpy().reshape(-1,1))
    
        # Mean if score conditional of W, X, A, and B (U-Statistics Kernel)
        sbar_n = (1/(N*M))*(R.to_numpy().T @ (hbar_n - e_n).to_numpy().reshape(-1,1))
    
        # First projections
        # s1c_n
        t1 = np.vstack([W, np.zeros((N,)), np.zeros((N,)), np.ones((N,))])
        t2 = (1 - pi_x) * (1/n)*np.exp(theta[0]*W + theta[3]) * \
                         (A - 1/(1 +(1/n)*np.exp(theta[0]*W + theta[3])))
        t3 = np.vstack([W, np.ones((N,)), W, np.ones((N,))])
        t4 = pi_x       * (1/n)*np.exp(theta[0]*W + theta[1] + theta[2]*W + theta[3]) * \
                         (A - 1/(1 +(1/n)*np.exp(theta[0]*W + theta[1] + theta[2]*W + theta[3]))) 
        s1c_n = (t1*t2 + t3*t4).T
    
        # s1p_n
        t1 = np.vstack([np.zeros((M,)), X, np.zeros((M,)), np.ones((M,))])
        t2 = (1 - pi_w) * (1/n)*np.exp(theta[1]*X + theta[3]) * \
                          (B - 1/(1 +(1/n)*np.exp(theta[1]*X + theta[3]))) 
        t3 = np.vstack([np.ones((M,)), X, X, np.ones((M,))])
        t4 = pi_w       * (1/n)*np.exp(theta[0] + theta[1]*X + theta[2]*X + theta[3]) * \
                          (B - 1/(1 +(1/n)*np.exp(theta[0] + theta[1]*X + theta[2]*X + theta[3])))
        s1p_n = (t1*t2 + t3*t4).T
    
        # Construct components of score decomposition (population values)
        S_n[s,(b*4):((b+1)*4)]  = s_n.T - b_n
        U1_n[s,(b*4):((b+1)*4)] = np.mean(s1c_n - b_n, axis=0) + np.mean(s1p_n - b_n, axis=0)
        U2_n[s,(b*4):((b+1)*4)] = (sbar_n.T - b_n) - U1_n[s,(b*4):((b+1)*4)]
        V_n[s,(b*4):((b+1)*4)]  = s_n.T - sbar_n.T
    
        #----------------------------------------------------------#
        #- STEP 3 : COMPUTE PSEUDO COMPOSITE LIKELIHOOD ESTIMATES -#
        #----------------------------------------------------------#
    
        #--------------------------------------------------------------------------#
        #- Estimation is with the bilogit() command included in the netrics module-#
        #--------------------------------------------------------------------------#
        
        # (i) Dense network standard errors
        # ---------------------------------
        
        [theta_BL, vcov_theta_BL]= netrics.bilogit(Y, R, nocons=True, silent=True, cov='dense')
    
        # Adjust intercept for -ln(n) in theoretical approximating sequence
        theta_BL[3,0] = theta_BL[3,0] + np.log(n)
    
        # Save pseudo composite MLE of interaction coefficient
        theta_hat[s,b] = theta_BL[2,0]
    
        # See if true interaction coefficient is inside dense Wald-based confidence interval
        coverage[s,b*4]  = (theta[2]<=theta_BL[2,0] + 1.96*np.sqrt(vcov_theta_BL[2,2]))*\
                           (theta[2]>=theta_BL[2,0] - 1.96*np.sqrt(vcov_theta_BL[2,2]))
           
        # Standard error length
        se_hat[s,b*4]    = np.sqrt(vcov_theta_BL[2,2])
    
        # (ii) Bias correction / Sparse network standard errors
        # -----------------------------------------------------
        
        [theta_BL, vcov_theta_BL]= netrics.bilogit(Y, R, nocons=True, silent=True, cov='sparse')
  
        # Adjust intercept for -ln(n) in theoretical approximating sequence
        theta_BL[3,0] = theta_BL[3,0] + np.log(n)
    
        # See if true interaction coefficient is inside sparse Wald-based confidence interval
        coverage[s,b*4+1]  = (theta[2]<=theta_BL[2,0] + 1.96*np.sqrt(vcov_theta_BL[2,2]))*\
                             (theta[2]>=theta_BL[2,0] - 1.96*np.sqrt(vcov_theta_BL[2,2]))
               
        # Standard error length
        se_hat[s,b*4+1] = np.sqrt(vcov_theta_BL[2,2])
    
        end = time.time()
        if (s+1) % 500 == 0:
            print("Time required f/ MC rep  " + str(s+1) + " of " + str(S) + ": " + str(end-start))      

    #-------------------------------------------------------------------------#
    #- STEP 4 : ASSESS QUALITY OF SPARSE NETWORK APPROXIMATION THEORY        -#
    #-------------------------------------------------------------------------#     
    
    # Compute variance of (population) score components by Monte Carlo integration
    Var_U1  = np.cov(U1_n[:,(b*4):((b+1)*4)] , rowvar = False, ddof = 1)
    Var_U1V = np.cov(U1_n[:,(b*4):((b+1)*4)] + V_n[:,(b*4):((b+1)*4)], rowvar = False, ddof = 1)

    # Population asymptotic variance under dense and sparse regimes
    vcov_theta_dense  = (iGAMMA @ Var_U1 @ iGAMMA)*(n**2)
    vcov_theta_sparse = (iGAMMA @ Var_U1V @ iGAMMA)*(n**2)

    # (Oracle) Coverage - dense case
    coverage[:,b*4+2]  = (theta[2]<=theta_hat[:,b] + 1.96*np.sqrt(vcov_theta_dense[2,2]))*\
                         (theta[2]>=theta_hat[:,b] - 1.96*np.sqrt(vcov_theta_dense[2,2]))

    # (Oracle) Standard error length - dense case
    se_hat[:,b*4+2]      = np.sqrt(vcov_theta_dense[2,2])
    
    # (Oracle) Coverage - sparse case
    coverage[:,b*4+3]  = (theta[2]<=theta_hat[:,b] + 1.96*np.sqrt(vcov_theta_sparse[2,2]))*\
                         (theta[2]>=theta_hat[:,b] - 1.96*np.sqrt(vcov_theta_sparse[2,2]))

    se_hat[:,b*4+3]    = np.sqrt(vcov_theta_sparse[2,2])

Monte Carlo design 1 of 5
Number of consumers and products, n = N + M : 64
Marginal purchase probability               : 0.080
Bias in pseudo-score vector n^(3/2)*S_n     : 3.766
(Above bias is for the component of S_n corresponding to the interaction coefficient)
Time required f/ MC rep  500 of 5000: 0.021091699600219727
Time required f/ MC rep  1000 of 5000: 0.01592230796813965
Time required f/ MC rep  1500 of 5000: 0.015247821807861328
Time required f/ MC rep  2000 of 5000: 0.015500783920288086
Time required f/ MC rep  2500 of 5000: 0.015883922576904297
Time required f/ MC rep  3000 of 5000: 0.015332698822021484
Time required f/ MC rep  3500 of 5000: 0.01657891273498535
Time required f/ MC rep  4000 of 5000: 0.027230024337768555
Time required f/ MC rep  4500 of 5000: 0.016371965408325195
Time required f/ MC rep  5000 of 5000: 0.017635107040405273
Monte Carlo design 2 of 5
Number of consumers and products, n = N + M : 144
Marginal purchase probability               : 0.036
Bias in ps

### Monte Carlo Results

The next few blocks of code print out the Monte Carlo simulation results which appear in **Tables 3** and **4** of the main paper (in Appendix C). First I print out the mean and median bias of the estimated interaction coefficient, $\hat{\beta}_{wx}$. This is in rows 1 and 2 below, with the different samples sizes corresponding to each column (from smallest to largest). I also compute the standard deviation of $\hat{\beta}_{wx}$ across the $5,000$ Monte Carlo replications. I use both the sample standard deviation (row 3) and a robust measure (row 4). The two estimates a very close except for the smallest sample size. The paper reports only the robust measure. The mapping of the numbers printed out below to the tables in the paper is specified in the printed output.

In [17]:
from scipy.stats import norm

np.set_printoptions(suppress=True, precision=5)
print("")
print("Population value of interaction coefficient : " + '%.5f' % theta[2])
print("")
print("Mean bias of point estimates, Table 3, Row 1")
print(np.mean(theta_hat-theta[2],axis=0))
print("")
print("Median bias of point estimates, Table 3, Row 2")
print(np.median(theta_hat-theta[2],axis=0))
print("")
print("Moment-based standard deviation of point estimates, not reported in paper")
print(np.std(theta_hat,axis=0))
print("")
print("Quantile-based estimate of standard deviation of point estimates, Table 3, Row 3")
print((np.quantile(theta_hat, q=0.95, axis=0)-np.quantile(theta_hat, q=0.05, axis=0))/(norm.ppf(0.95)-norm.ppf(0.05)))  


Population value of interaction coefficient : 1.38629

Mean bias of point estimates, Table 3, Row 1
[0.12093 0.06152 0.03957 0.01712 0.01186]

Median bias of point estimates, Table 3, Row 2
[0.16322 0.06349 0.04063 0.01489 0.01271]

Moment-based standard deviation of point estimates, not reported in paper
[1.36191 0.42632 0.30379 0.20163 0.15037]

Quantile-based estimate of standard deviation of point estimates, Table 3, Row 3
[0.70386 0.42214 0.29684 0.19723 0.15162]


Next I report coverage estimates of the various confidence intervals. The first row reports the coverage of dense intervals for each design. The second reports the coverage of sparse intervals. These intervals are computed by the **bilogit()** command in the **netrics** module as described in an appendix to the paper. The final two rows report the coverage of infeasible dense and sparse oracle intervals (not reported in the paper).

In [19]:
print(np.mean(coverage,axis=0).reshape((NumDesigns,4)).T)
print("")
print("NOTES: Row 1 above appears as Row 6 of Table 3")
print("NOTES: Row 2 above appears as Row 5 of Table 3")

[[0.3468 0.362  0.3506 0.3208 0.2922]
 [0.8754 0.9286 0.9442 0.9496 0.9434]
 [0.475  0.5    0.5186 0.5206 0.5294]
 [0.9266 0.9638 0.9716 0.9744 0.9792]]

NOTES: Row 1 above appears as Row 6 of Table 3
NOTES: Row 2 above appears as Row 5 of Table 3


Next I report mean estimated standard errors for $\hat{\beta}_{wx}$. Results are organized in the same order as those for confidence intervals above.

In [20]:
print(np.mean(se_hat,axis=0).reshape((NumDesigns,4)).T)
print("")
print("NOTES: Row 2 above appears as Row 4 of Table 3")

[[0.17713 0.10932 0.07399 0.04429 0.02964]
 [0.59686 0.39886 0.29754 0.19923 0.14896]
 [0.22501 0.14637 0.11007 0.07411 0.05451]
 [0.67789 0.46378 0.34453 0.23402 0.1783 ]]

NOTES: Row 2 above appears as Row 4 of Table 3


### Population score decomposition

The asymptotic theory in the paper hinges critically on the behavior of the "score" vector under sparse sequences. This next block of code prints out some basic summary statistics for the components of this "score". These statistics are reported in **Table 4** of the paper (in Appendix C).

In [23]:
tS_n    = S_n / np.diag(np.cov(S_n, rowvar = False, ddof = 1))**(1/2)
tU1_n   = U1_n / np.diag(np.cov(U1_n, rowvar = False, ddof = 1))**(1/2)
tU2_n   = U2_n / np.diag(np.cov(U2_n, rowvar = False, ddof = 1))**(1/2)
tV_n    = V_n / np.diag(np.cov(V_n, rowvar = False, ddof = 1))**(1/2)
tU1V_n  = (U1_n + V_n) / np.diag(np.cov(U1_n + V_n, rowvar = False, ddof = 1))**(1/2)

np.set_printoptions(suppress=False, precision=5)

#------------------------------------------------#
#- CASE 1: n=256 design                         -#
#------------------------------------------------#

print(" ")
n=256

print("Mean and standard deviations of scaled score components: S_n, U1_n, U2_n, V_n, U1n+V_n, n = 256 design")
print("These results appear in Rows 1 & 2 of Table 4")
print("Note: The Row 1, Column 1 result reported in Table 4 correspond to the first number below plus the analytic")
print("      bias number calculated above and reported in Row, Column 6 of Table 4")
print(n**(3/2)*np.mean(np.vstack([S_n[:,(NumDesigns-3)*4+2], U1_n[:,(NumDesigns-3)*4+2], U2_n[:,(NumDesigns-3)*4+2], \
                                  V_n[:,(NumDesigns-3)*4+2],  \
                                  U1_n[:,(NumDesigns-3)*4+2] + V_n[:,(NumDesigns-3)*4+2]]).T, axis=0))

print(n**(3/2)*np.diag(np.cov(np.vstack([S_n[:,(NumDesigns-3)*4+2], U1_n[:,(NumDesigns-3)*4+2], U2_n[:,(NumDesigns-3)*4+2], \
                                         V_n[:,(NumDesigns-3)*4+2], \
                                         U1_n[:,(NumDesigns-3)*4+2] + V_n[:,(NumDesigns-3)*4+2]]).T, \
                              rowvar = False, ddof = 1))**(1/2))

print(" ")
t = np.vstack([tS_n[:,(NumDesigns-3)*4+2], tU1_n[:,(NumDesigns-3)*4+2],  tU2_n[:,(NumDesigns-3)*4+2], \
               tV_n[:,(NumDesigns-3)*4+2], tU1V_n[:,(NumDesigns-3)*4+2]]).T

print("Tail frequences on standardized score components")
print("These results appear in Rows 3 to 6 of Table 4")
print(np.mean(t>=1.6449, axis = 0))
print(np.mean(t<=-1.6449, axis = 0))
print(np.mean(t>=1.96, axis = 0))
print(np.mean(t<=-1.96, axis = 0))

#------------------------------------------------#
#- CASE 2: n=1024 design                        -#
#------------------------------------------------#

print(" ")
n=1024

print("Mean and standard deviations of scaled score components: S_n, U1_n, U2_n, V_n, U1n+V_n, n = 1024 design")
print("These results appear in Rows 7 & 8 of Table 4")
print("Note: The Row 7, Column 1 result reported in Table 4 correspond to the first number below plus the analytic")
print("      bias number calculated above and reported in Row, Column 6 of Table 4")
print(n**(3/2)*np.mean(np.vstack([S_n[:,(NumDesigns-1)*4+2], U1_n[:,(NumDesigns-1)*4+2], U2_n[:,(NumDesigns-1)*4+2], \
                                  V_n[:,(NumDesigns-1)*4+2],  \
                                  U1_n[:,(NumDesigns-1)*4+2] + V_n[:,(NumDesigns-1)*4+2]]).T, axis=0))

print(n**(3/2)*np.diag(np.cov(np.vstack([S_n[:,(NumDesigns-1)*4+2], U1_n[:,(NumDesigns-1)*4+2], U2_n[:,(NumDesigns-1)*4+2], \
                                         V_n[:,(NumDesigns-1)*4+2], \
                                         U1_n[:,(NumDesigns-1)*4+2] + V_n[:,(NumDesigns-1)*4+2]]).T, \
                              rowvar = False, ddof = 1))**(1/2))

print(" ")
t = np.vstack([tS_n[:,(NumDesigns-1)*4+2], tU1_n[:,(NumDesigns-1)*4+2],  tU2_n[:,(NumDesigns-1)*4+2], \
               tV_n[:,(NumDesigns-1)*4+2], tU1V_n[:,(NumDesigns-1)*4+2]]).T

print("Tail frequences on standardized score components")
print("These results appear in Rows 9 t0 12 of Table 4")
print(np.mean(t>=1.6449, axis = 0))
print(np.mean(t<=-1.6449, axis = 0))
print(np.mean(t>=1.96, axis = 0))
print(np.mean(t<=-1.96, axis = 0))

 
Mean and standard deviations of scaled score components: S_n, U1_n, U2_n, V_n, U1n+V_n, n = 256 design
These results appear in Rows 1 & 2 of Table 4
Note: The Row 1, Column 1 result reported in Table 4 correspond to the first number below plus the analytic
      bias number calculated above and reported in Row, Column 6 of Table 4
[ 0.06278  0.04457 -0.00445  0.02266  0.06723]
[5.2165  3.84598 0.31958 3.61217 5.20898]
 
Tail frequences on standardized score components
These results appear in Rows 3 to 6 of Table 4
[0.0578 0.0546 0.0422 0.0542 0.0576]
[0.04   0.0432 0.0502 0.0472 0.0412]
[0.0324 0.029  0.0282 0.0308 0.0304]
[0.0154 0.0184 0.036  0.0246 0.0166]
 
Mean and standard deviations of scaled score components: S_n, U1_n, U2_n, V_n, U1n+V_n, n = 1024 design
These results appear in Rows 7 & 8 of Table 4
Note: The Row 7, Column 1 result reported in Table 4 correspond to the first number below plus the analytic
      bias number calculated above and reported in Row, Column 6 of Ta

### Wrapping Up

There is clearly a tremendous amount of additional research that could be done. Both in terms of exploring the accuracy of dense versus sparse asymptotic approximations across different designs and also in terms of evaluating the properties of different approaches/methods of interference. What is reported above represents just an initial pass. The results are sufficiently promising to suggest that sparse network asymptotics may be useful in practice.